In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

zip_path = "/content/drive/MyDrive/plate-project/archive.zip"
extract_path = "/content/yolo_plate_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Zip açıldı:", os.listdir(extract_path))

Zip açıldı: ['label', 'images']


In [3]:
import cv2
import glob

IMG_SIZE = 224
image_paths = glob.glob("/content/yolo_plate_dataset/images/*")

for path in image_paths:
    img = cv2.imread(path)
    if img is None:
        continue
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    cv2.imwrite(path, img_resized)

In [2]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
import os
import cv2
import albumentations as A
import matplotlib.pyplot as plt
from glob import glob

# === Klasörler ===
IMG_DIR = "/content/yolo_plate_dataset/images"
LABEL_DIR = "/content/yolo_plate_dataset/label"
SAVE_IMG_DIR = "/content/yolo_plate_dataset/images_aug"
SAVE_LABEL_DIR = "/content/yolo_plate_dataset/label_aug"

os.makedirs(SAVE_IMG_DIR, exist_ok=True)
os.makedirs(SAVE_LABEL_DIR, exist_ok=True)

# === Augmentations ===
augmentations = [
    ("Rotate", A.Rotate(limit=20, p=1.0)),
    ("HorizontalFlip", A.HorizontalFlip(p=1.0)),
    ("Brightness", A.RandomBrightnessContrast(p=1.0)),
    ("Zoom", A.RandomScale(scale_limit=0.4, p=1.0)),
    ("GaussNoise", A.GaussNoise(var_limit=(10.0, 50.0), p=1.0)),
    ("Cutout", A.CoarseDropout(max_holes=5, max_height=20, max_width=20, p=1.0)),
    ("MotionBlur", A.MotionBlur(p=1.0)),
    ("MedianBlur", A.MedianBlur(blur_limit=3, p=1.0))
]

# === Tüm resimler üzerinde dön ===
image_paths = sorted(glob(os.path.join(IMG_DIR, "*.jpg")))

for image_path in image_paths:
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    label_path = os.path.join(LABEL_DIR, base_name + ".txt")

    if not os.path.exists(label_path):
        print(f"Etiket bulunamadı: {label_path}")
        continue

    # Resmi oku
    image = cv2.imread(image_path)
    if image is None:
        print(f"Hatalı resim: {image_path}")
        continue
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Etiketi oku
    bboxes = []
    class_labels = []
    with open(label_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:])
            bboxes.append([x_center, y_center, width, height])
            class_labels.append(class_id)

    # Her augment için dön
    for title, aug in augmentations:
        transform = A.Compose([aug], bbox_params=A.BboxParams(format='yolo', label_fields=["class_labels"]))
        try:
            augmented = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        except Exception as e:
            print(f"Hata {title} - {base_name}: {e}")
            continue

        image_aug = augmented["image"]
        bboxes_aug = augmented["bboxes"]

        if len(bboxes_aug) == 0:
            print(f"Skip {base_name}_{title}: bbox kayboldu.")
            continue

        # Kaydet
        save_img_path = os.path.join(SAVE_IMG_DIR, f"{base_name}_{title}.jpg")
        save_label_path = os.path.join(SAVE_LABEL_DIR, f"{base_name}_{title}.txt")
        cv2.imwrite(save_img_path, cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR))
        with open(save_label_path, "w") as f:
            for bbox in bboxes_aug:
                f.write(f"0 {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")

        # Görselleştir
        h, w, _ = image_aug.shape
        img_vis = image_aug.copy()
        for bbox in bboxes_aug:
            x_center, y_center, width, height = bbox
            x1 = int((x_center - width / 2) * w)
            y1 = int((y_center - height / 2) * h)
            x2 = int((x_center + width / 2) * w)
            y2 = int((y_center + height / 2) * h)
            cv2.rectangle(img_vis, (x1, y1), (x2, y2), (255, 0, 0), 2)



/tmp/ipython-input-5-1909207485.py:22: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  ("GaussNoise", A.GaussNoise(var_limit=(10.0, 50.0), p=1.0)),
/tmp/ipython-input-5-1909207485.py:23: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  ("Cutout", A.CoarseDropout(max_holes=5, max_height=20, max_width=20, p=1.0)),
/usr/local/lib/python3.11/dist-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


Skip 1000_Cutout: bbox kayboldu.


/usr/local/lib/python3.11/dist-packages/albumentations/augmentations/dropout/functional.py:559: RuntimeWarning: invalid value encountered in divide
  visibility_ratios = remaining_areas / box_areas


Skip 779_Cutout: bbox kayboldu.
Hata Rotate - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata HorizontalFlip - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata Brightness - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata Zoom - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata GaussNoise - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata Cutout - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata MotionBlur - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata MedianBlur - 823: y_max is less than or equal to y_min for bbox [0.611253 0.691324 0.612237 0.691324 0.      ].
Hata Rotate - 829: y_max is less than or e

In [6]:
import os
import random
import shutil
from glob import glob

base_dir = "/content/yolo_plate_dataset"

# 1. Tüm görselleri topla (orijinal + augment)
all_image_paths = glob(f"{base_dir}/images/*.jpg") + glob(f"{base_dir}/images_aug/*.jpg")
random.shuffle(all_image_paths)

# 2. Eğitim ve doğrulama oranı
train_size = int(0.8 * len(all_image_paths))
train_imgs = all_image_paths[:train_size]
val_imgs = all_image_paths[train_size:]

# 3. Hedef klasörleri oluştur
for split in ['train', 'val']:
    os.makedirs(f"{base_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{base_dir}/labels/{split}", exist_ok=True)

# 4. Dosyaları taşı
def move_data(img_paths, split):
    for img_path in img_paths:
        img_name = os.path.basename(img_path)
        label_name = img_name.replace(".jpg", ".txt")

        # Görseli taşı
        shutil.copy(img_path, f"{base_dir}/images/{split}/{img_name}")

        # Etiket hangi klasördeyse onu bul
        label_src = None
        for lbl_dir in ['label', 'label_aug']:
            possible_path = os.path.join(base_dir, lbl_dir, label_name)
            if os.path.exists(possible_path):
                label_src = possible_path
                break

        if label_src:
            shutil.copy(label_src, f"{base_dir}/labels/{split}/{label_name}")
        else:
            print(f"⚠️ Etiket bulunamadı: {label_name}")

# 5. Taşıma işlemi
move_data(train_imgs, 'train')
move_data(val_imgs, 'val')

In [ ]:
model.train(
    data="/content/yolo_plate_dataset/data.yaml",
    epochs=25,
    patience=4,
    imgsz=224,
    batch=8,
    save=True,                      # model kaydetsin
    save_period=1,                  # her epochta kaydetsin
    resume=False,                   # sıfırdan başla
    project="/content/drive/MyDrive/yolo_training",  # Drive klasörü
    name="plate_model",             # Kaydedilecek alt klasör
    exist_ok=True                   # varsa üzerine yazsın
)

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_plate_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=plate_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=4, perspective=0.0, plots=True, pose=12.0

train: Scanning /content/yolo_plate_dataset/labels/train... 116210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 116210/116210 [01:14<00:00, 1565.58it/s]


train: New cache created: /content/yolo_plate_dataset/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 286.6±327.0 MB/s, size: 29.4 KB)


val: Scanning /content/yolo_plate_dataset/labels/val... 30914 images, 0 backgrounds, 0 corrupt: 100%|██████████| 30914/30914 [00:19<00:00, 1565.29it/s]


val: New cache created: /content/yolo_plate_dataset/labels/val.cache
Plotting labels to /content/drive/MyDrive/yolo_training/plate_model/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to /content/drive/MyDrive/yolo_training/plate_model
Starting training for 25 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      18/25     0.352G     0.7886     0.4008     0.8342          2        224: 100%|██████████| 14527/14527 [17:19<00:00, 13.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1933/1933 [02:03<00:00, 15.67it/s]


                   all      30914      32684      0.979       0.92       0.96      0.764

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      19/25     0.352G     0.7721     0.3906     0.8309          2        224: 100%|██████████| 14527/14527 [17:12<00:00, 14.07it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1933/1933 [02:03<00:00, 15.70it/s]


                   all      30914      32684      0.979       0.92      0.961      0.769

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/25     0.352G     0.7542     0.3812     0.8277          8        224:  90%|████████▉ | 13018/14527 [15:28<01:45, 14.34it/s]

In [5]:
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/yolo_training/plate_model/weights/best.pt")
model.export(format="onnx", dynamic=True)

Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolo_training/plate_model/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 5, 1029) (11.7 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59', 'onnxruntime'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.1s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.61...
ONNX: export success ✅ 8.3s, saved as '/content/drive/MyDrive/yolo_training/plate_model/weights/best.onnx' (11.6 MB)

Export complete (8.8s)
Results saved to /content/drive

'/content/drive/MyDrive/yolo_training/plate_model/weights/best.onnx'